Taller 1: procesamiento de lenguaje natural

Andrés Rodriguez

David Orozco

Javier Yela

Alberto Torres

# Analisis de sentimientos en dataset de wits

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Ohtar10/icesi-nlp/blob/main/Sesion1/7-sentiment-analysis.ipynb)

El taller propone el análisis de un dataset tomado de Kaggle el cual contiene más de 1 millón de registros. Cada resgistro contiene las columnas target, ID, date, flag, user y text. El dataset fue recortado para mejorar su procesamiento. Se tomó una muestra aleatoria de 5000 twitts.

El dataset resultante fue mapeado para que el dominio de la variable target (0,4) fuera reemplazado por (neg, pos). Esto con el fin de facilitar la interpretación de los resultados.

#Importación de librerías y puesta en marcha del entorno

In [ ]:
import pkg_resources
import warnings

warnings.filterwarnings('ignore')

installed_packages = [package.key for package in pkg_resources.working_set]
IN_COLAB = 'google-colab' in installed_packages

/tmp/ipython-input-2396000874.py:1: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources


In [ ]:
!test '{IN_COLAB}' = 'True' && wget  https://github.com/Ohtar10/icesi-nlp/raw/refs/heads/main/requirements.txt && pip install -r requirements.txt

--2025-08-14 01:07:25--  https://github.com/Ohtar10/icesi-nlp/raw/refs/heads/main/requirements.txt
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Ohtar10/icesi-nlp/refs/heads/main/requirements.txt [following]
--2025-08-14 01:07:25--  https://raw.githubusercontent.com/Ohtar10/icesi-nlp/refs/heads/main/requirements.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 349 [text/plain]
Saving to: ‘requirements.txt’

requirements.txt    100%[===================>]     349  --.-KB/s    in 0s      

2025-08-14 01:07:25 (6.29 MB/s) - ‘requirements.txt’ saved [349/349]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!test '{IN_COLAB}' = 'True' && wget  https://github.com/Ohtar10/icesi-nlp/raw/refs/heads/main/Sesion1/moviereviews.tsv

#Carga del dataset

In [ ]:
import pandas as pd
import numpy as np

#reviews = pd.read_csv('/content/moviereviews.tsv', sep='\t')
reviews = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', encoding='latin-1', )
column_names = ['sentiment', 'id', 'date', 'query', 'user', 'text']
reviews.columns = column_names
reviews = reviews.sample(n=5000, random_state=42) # Added random state for reproducibility
reviews['sentiment'] = reviews['sentiment'].map({0: 'neg', 4: 'pos'}) #mapeo de etiquetas
#reviews = reviews[0:5000]
reviews.head()

,sentiment,id,date,query,user,text
541200,neg,2200003313,Tue Jun 16 18:18:13 PDT 2009,NO_QUERY,DEWGetMeTho77,@Nkluvr4eva My poor little dumpling In Holmde...
750,neg,1467998601,Mon Apr 06 23:11:18 PDT 2009,NO_QUERY,Young_J,I'm off too bed. I gotta wake up hella early t...
766711,neg,2300049112,Tue Jun 23 13:40:12 PDT 2009,NO_QUERY,dougnawoschik,I havent been able to listen to it yet My spe...
285055,neg,1993474319,Mon Jun 01 10:26:09 PDT 2009,NO_QUERY,thireven,now remembers why solving a relatively big equ...
705995,neg,2256551006,Sat Jun 20 12:56:51 PDT 2009,NO_QUERY,taracollins086,"Ate too much, feel sick"


#Preprocesamiento de datos

Se eliminan valores nulos

In [ ]:
reviews.dropna(inplace=True)
reviews.text = reviews.text.apply(lambda r: r.strip())
blanks = reviews[reviews.text == ''].index
reviews.drop(blanks, inplace=True)

In [ ]:
reviews[reviews.text == ''].index

Index([], dtype='int64')

Primer análisis: balance del conjunto de datos

In [ ]:
reviews.sentiment.value_counts()

,count
sentiment,
pos,2504
neg,2496


El dataset se encuantr bien balanceado con un poco más de 2.500 twitts con etiqueta positiva y casi 2.500 twitts con etiqueta negativa.

#VADER para procesamiento de lenguaje natural y análisis de emociones

In [ ]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()
reviews['scores'] = reviews.text.apply(lambda r: sid.polarity_scores(r))
reviews.head()

,sentiment,id,date,query,user,text,scores
541200,neg,2200003313,Tue Jun 16 18:18:13 PDT 2009,NO_QUERY,DEWGetMeTho77,@Nkluvr4eva My poor little dumpling In Holmde...,"{'neg': 0.144, 'neu': 0.744, 'pos': 0.112, 'co..."
750,neg,1467998601,Mon Apr 06 23:11:18 PDT 2009,NO_QUERY,Young_J,I'm off too bed. I gotta wake up hella early t...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
766711,neg,2300049112,Tue Jun 23 13:40:12 PDT 2009,NO_QUERY,dougnawoschik,I havent been able to listen to it yet My spe...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
285055,neg,1993474319,Mon Jun 01 10:26:09 PDT 2009,NO_QUERY,thireven,now remembers why solving a relatively big equ...,"{'neg': 0.168, 'neu': 0.711, 'pos': 0.122, 'co..."
705995,neg,2256551006,Sat Jun 20 12:56:51 PDT 2009,NO_QUERY,taracollins086,"Ate too much, feel sick","{'neg': 0.452, 'neu': 0.548, 'pos': 0.0, 'comp..."


Con los puntajes obtenidos después de ser procesado el texto con VADER, podemos obtener un resultado en una etiqueta de predicción:

In [ ]:
reviews['compound'] = reviews.scores.apply(lambda s: s['compound'])
reviews['prediction'] = reviews['compound'].apply(lambda c: 'pos' if c > 0 else 'neg')
reviews.head()

,sentiment,id,date,query,user,text,scores,compound,prediction
541200,neg,2200003313,Tue Jun 16 18:18:13 PDT 2009,NO_QUERY,DEWGetMeTho77,@Nkluvr4eva My poor little dumpling In Holmde...,"{'neg': 0.144, 'neu': 0.744, 'pos': 0.112, 'co...",-0.4013,neg
750,neg,1467998601,Mon Apr 06 23:11:18 PDT 2009,NO_QUERY,Young_J,I'm off too bed. I gotta wake up hella early t...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,neg
766711,neg,2300049112,Tue Jun 23 13:40:12 PDT 2009,NO_QUERY,dougnawoschik,I havent been able to listen to it yet My spe...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,neg
285055,neg,1993474319,Mon Jun 01 10:26:09 PDT 2009,NO_QUERY,thireven,now remembers why solving a relatively big equ...,"{'neg': 0.168, 'neu': 0.711, 'pos': 0.122, 'co...",-0.2263,neg
705995,neg,2256551006,Sat Jun 20 12:56:51 PDT 2009,NO_QUERY,taracollins086,"Ate too much, feel sick","{'neg': 0.452, 'neu': 0.548, 'pos': 0.0, 'comp...",-0.5106,neg


#Métricas del modelo

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

y_true = reviews.sentiment.values
y_pred = reviews.prediction.values

acc = accuracy_score(y_true, y_pred)
cm = confusion_matrix(y_true, y_pred)
cr = classification_report(y_true, y_pred)


print(f"Accuracy:\n{acc}\n")
print(f"Classification Report:\n{cr}")
print(f"Confusion Matrix:\n{cm}")

Accuracy:
0.6538

Classification Report:
              precision    recall  f1-score   support

         neg       0.64      0.70      0.67      2496
         pos       0.67      0.61      0.64      2504

    accuracy                           0.65      5000
   macro avg       0.65      0.65      0.65      5000
weighted avg       0.65      0.65      0.65      5000

Confusion Matrix:
[[1736  760]
 [ 971 1533]]


Exactitud:

El modelo predijo con una exactitud del 65.38%, siendo este un desempeño modesto. El modelo tiene un margen de mejora considerable.

Reporte de clasificación:

El modelo acierta la predicción "negativa" con un 64%, mientras que predice algo "positivo" con un 65%.

El modelo fue capaz de identificar correctamente el 70% de todos los casos que eran realmente "negativos". Por otra parte, el modelo solo pudo identificar el 61% de todos los casos que eran realmente "positivos". En pocas palabras: El modelo es notablemente mejor para encontrar instancias negativas que positivas.

Matriz de confusión

Verdaderos Negativos (TN): 1736: El modelo predijo correctamente 1736 muestras como negativas.

Falsos Positivos (FP): 760: El modelo predijo incorrectamente 760 muestras como positivas (en realidad eran negativas). Error de Tipo I.

Falsos Negativos (FN): 971: El modelo predijo incorrectamente 971 muestras como negativas (en realidad eran positivas). Error de Tipo II.

Verdaderos Positivos (TP): 1533: El modelo predijo correctamente 1533 muestras como positivas.

En conclusión, el modelo tiene un rendimiento modesto de predicción. Su punto fuerte es la prediccón de sentimientos negativos, mientras que su punto más débil es la predicción de los sentimientos positivos.